# Importing data

In [9]:
import pandas as pd
import numpy as np
import sqlite3

In [10]:
db = sqlite3.connect('winequal.db')
cur = db.cursor()


cur.execute("ALTER TABLE red ADD COLUMN color INT DEFAULT 0")
cur.execute("ALTER TABLE white ADD COLUMN color INT DEFAULT 1")

cur.execute('SELECT * from red')
red_data = cur.fetchall()

db.commit()
db.close()

db = sqlite3.connect('winequal.db')
cur = db.cursor()

cur.execute('SELECT * from white')
white_data = cur.fetchall()

db.commit()
db.close()

 

In [11]:
df = pd.DataFrame(red_data, columns=red_data[0])
df = df.drop(df.index[0])
df.head()
red = df.rename(columns={0:'white?'});
red.reset_index(drop=True, inplace=True)

red2=pd.DataFrame()
for key in red.keys():
    red2[key]=pd.to_numeric(red[key])

red2.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,white?
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


In [12]:
df2 = pd.DataFrame(white_data, columns=white_data[0])
df2 = df2.drop(df2.index[0])
white = df2.rename(columns={1:'white?'});
white.reset_index(drop=True, inplace=True)

white2=pd.DataFrame()
for key in white.keys():
    white2[key]=pd.to_numeric(white[key])


In [13]:
comp_data = pd.concat([white2, red2])
comp_data.reset_index(drop=True, inplace=True)

# Exploring data

In [14]:
from bokeh.charts import Histogram
from bokeh.charts import Bar
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.charts import BoxPlot

output_notebook()

Loading BokehJS ...

In [15]:

p1 =Histogram(comp_data, values='quality', color='white?',tools='pan, reset, box_zoom')
p1.xaxis.minor_tick_line_color=None

p2 = Bar(comp_data, values='quality', label='white?', agg='count',tools='pan, reset, box_zoom', legend=None)

p = gridplot([[p2,p1]], plot_width=400, plot_height=400)
show(p)

# In depth with each parameter

In [16]:
import itertools

In [17]:
red_y = np.array([])
white_y = np.array([])
red_x = np.array([])
white_x = np.array([])

plot_list = list()

key_subset = comp_data.keys().drop(['quality', 'white?'])

for key_sub in key_subset:
    red_y = np.array([])
    white_y = np.array([])
    red_x = np.array([])
    white_x = np.array([])
    for i,j in itertools.product(comp_data['white?'].unique(), comp_data['quality'].unique()):
        subset=comp_data[(comp_data['white?']==i)&(comp_data['quality']==j)]
        #print(subset.head())
        if i==1:
            mean=subset[key_sub].mean()
            white_y=np.append(white_y,mean)
            white_x=np.append(white_x,j)
        if i==0:
            mean=subset[key_sub].mean()
            red_y=np.append(red_y,mean)
            red_x=np.append(red_x,j)

    sorted_white= np.sort([white_x, white_y])

    sorted_red = np.sort([red_x, red_y])

    TOOLS = 'hover, reset, crosshair'
    p = figure(plot_width=300, plot_height=300, tools=TOOLS)
    
    p.line(sorted_white[0], sorted_white[1], color = 'green', legend = 'white')
    p.line(sorted_red[0], sorted_red[1], color='red', legend = 'red')
    p.axis[0].axis_label='quality'
    p.axis[1].axis_label= key_sub
    p.title.text = 'mean %s vs. quality' %(key_sub)
    p.legend.location = 'top_left'
    
    plot_list.append(p)

p3=gridplot([plot_list[0:3], plot_list[3:6]])
show(p3)

In [18]:
p4=gridplot([plot_list[6:9], plot_list[9:12]])
show(p4)

# Examining correlation

In [19]:
subset_red = comp_data[comp_data['white?']==0][key_subset]
corr_red = subset_red.corr()

subset_white = comp_data[comp_data['white?']==1][key_subset]
corr_white = subset_white.corr()


In [20]:
corr_red[(abs(corr_red)>=0.5)&(abs(corr_red)<1)]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
fixed acidity,NaN,NaN,0.671703,NaN,NaN,NaN,NaN,0.668047,-0.682978,NaN,NaN
volatile acidity,NaN,NaN,-0.552496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
citric acid,0.671703,-0.552496,NaN,NaN,NaN,NaN,NaN,NaN,-0.541904,NaN,NaN
residual sugar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chlorides,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
free sulfur dioxide,NaN,NaN,NaN,NaN,NaN,NaN,0.667666,NaN,NaN,NaN,NaN
total sulfur dioxide,NaN,NaN,NaN,NaN,NaN,0.667666,NaN,NaN,NaN,NaN,NaN
density,0.668047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pH,-0.682978,NaN,-0.541904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sulphates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
corr_white[(abs(corr_white)>=0.5)&(abs(corr_white)<1)]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
fixed acidity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
volatile acidity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
citric acid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
residual sugar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.838966,NaN,NaN,NaN
chlorides,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
free sulfur dioxide,NaN,NaN,NaN,NaN,NaN,NaN,0.615501,NaN,NaN,NaN,NaN
total sulfur dioxide,NaN,NaN,NaN,NaN,NaN,0.615501,NaN,0.529881,NaN,NaN,NaN
density,NaN,NaN,NaN,0.838966,NaN,NaN,0.529881,NaN,NaN,NaN,-0.780138
pH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sulphates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Building Multiclass Classification Model for Quality

In [16]:
from sklearn import cross_validation
from sklearn import linear_model
from sklearn import metrics

In [1]:
xtrain, xtest, ytrain, ytest = cross_validation.train_test_split(subset_red, red2['quality'], train_size = 0.75)

NameError: name 'cross_validation' is not defined

In [24]:
rgr = linear_model.LogisticRegressionCV(solver='lbfgs',multi_class='multinomial', scoring='accuracy')
rgr.fit(xtrain, ytrain)

/Users/ericalonas/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found arrays with inconsistent numbers of samples: [   1 1199]

In [19]:
pred=rgr.predict(xtest)

In [20]:
metrics.accuracy_score(pred, ytest)

0.59250000000000003

In [49]:
from sklearn import svm

In [50]:
rgr2 = svm.SVC(kernel='rbf')
rgr2.fit(xtrain, ytrain)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [51]:
pred = rgr2.predict(xtest)
metrics.accuracy_score(ytest, pred)

0.57250000000000001

In [59]:
key_sub2 = xtrain.keys().drop(['residual sugar','fixed acidity','density'])

subset_xtrain = xtrain[key_sub2]
subset_xtest = xtest[key_sub2]

In [90]:
rgr3 = linear_model.LogisticRegressionCV(scoring='accuracy')
rgr3.fit(subset_xtrain, ytrain)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring='accuracy', solver='lbfgs', tol=0.0001,
           verbose=0)

In [91]:
pred3 = rgr3.predict(subset_xtest)

In [92]:
metrics.accuracy_score(ytest, pred3)

0.56999999999999995

## White wine modeling

In [104]:
xtrain, xtest, ytrain, ytest = cross_validation.train_test_split(subset_white, white2['quality'], train_size = 0.75)

In [108]:
rgr4 = linear_model.LogisticRegressionCV(solver='lbfgs',multi_class='multinomial')
rgr4.fit(xtrain, ytrain)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [109]:
pred=rgr4.predict(xtest)

In [110]:
metrics.accuracy_score(ytest, pred)

0.51755102040816325